# Thesis defense preparation 

I should identify the weaknesses in my work. What things are missing? What things are poorly explained? In which ways are my arguments poorly based?

Key weakpoints
- My conclusion that the target instrument might be possible because I numerically optimized something without simulating it might not be convincing
  - This is the elephant in the room
  - SIMULATE FOIL2 PG B!!!
    - Maybe simulate the 2 other identified interesting instruments
- My message about sample scattering off the detector and stuff is vague
  - Add graphs with integrated intensity over wavelengths
  - Add actual detector profiles for radii and wavelengths like those approximated semi-analytically
  - Try to work out corrections!
- I did not correct for wavelength spread in my fitting, which makes their meaning unclear
  - Work out correction
    - At least for tau, this should be easier
- It is unclear why my random search method should give close to optimal results
  - Add convergence graphs
  - Acknowledge that these should have been in an Appendix or so
    - Frame it as an example of a key challenge in writing a good report: you have to articulate every choice you made, however obvious to you from experience 

- Why simulate up to found constraints, what if by simulating you could find out you can do more than that?
  - There are soft and hard constraints in my model
  - Identified constraints are hard constraints like device field strenghts and 1 period on detector, verified by looking at plots and corresponding B range data.
    - Simulating further than these would mean simulating greater fields than possible given the device characteristics or measuring less than 1 full period for visibility, which is also not possible

# 1. $\lambda$-correction for $\tau$? Probably not a problem

Assuming a Gaussian $\lambda$ distribution, a more correct expression for $\sigma_t$ averaged over all wavelengths can be derived from the second moment of the distribution. $\tau \propto \lambda^2$, meaning that $E[\lambda^2]$ gives the right value. For a normal distribution $N(\lambda_0, \sigma^2)$ it can be derived that as $Var(\lambda) = \sigma^2 = E[\lambda^2] - E[\lambda]^2 = E[\lambda^2] - \lambda_0^2$, 
$$E[\lambda^2] = \lambda_0^2 + \sigma^2$$
This means that the true average $\bar{\tau}$ over all $\lambda$ is related to $\tau(\lambda_0)$ by
$$\frac{\bar{\tau}}{\tau(\lambda_0)} = 1 + \frac{\sigma^2}{\lambda_0^2}$$

In the most extreme case, $\Delta\lambda/\lambda_0 = 0.1$ which gives $\sigma/\lambda_0 = 0.042467$. This gives a correction factor of $1.001803$ or only about $0.2\%$. 

## 1.1 Effect on transmission $T$
More correctly, the average of the transmission $T = e^{-\tau}$ should be used. This can be seen to only further reduce the effect however.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy import integrate

for sigma in [0.0042467, 0.042467, 0.042467*2]: 
    x = np.linspace(1 - 5 * sigma, 1 + 5 * sigma, 100000)
    dx = x[1] - x[0]
    p = norm.pdf(x, 1, sigma)
    integrand = np.exp(-x**2) * p / np.exp(-1)
    plt.plot(x, p, linewidth=2)
    plt.plot(x, integrand)
    C = integrate.trapezoid(integrand, x = x)
    print(C)
    plt.show()


In [ ]:
from definitions import *
# Sample (R, t) [(m,m)]
samples = [(1e-6, 1e-3), (300e-9, 1e-3), (50e-9, 10e-3)]
Rs = [2e-6, 300e-9, 50e-9]
# Source (L0, DL) [(m,m)]
sources = [(4.321, 0.04321), (8, 0.8)]

# Source sample (thickness) pairing (L0, DL, R, t)
source_sample = [(4.321, 0.04321, 2e-6, 1e-3), (8, 0.8, 2e-6, 1e-3), (4.321, 0.04321, 300e-9, 10e-3), (8, 0.8, 300e-9, 5e-3), (4.321, 0.04321, 50e-9, 10e-3), (8, 0.8,50e-9, 10e-3)]

thickness_map = {
    (4.321, 2e-6):  1e-3,
    (8, 2e-6):  1e-3,
    (4.321, 300e-9): 10e-3,
    (8, 300e-9): 5e-3,
    (4.321, 50e-9): 10e-3,
    (8, 50e-9): 10e-3,
}
# Universal sample parameters (?)
phi = 0.015

delta_rho = 1.8e14 # 1/m^2 (?)
print(f"Constant parameters: drho = {delta_rho * 1e-14}e14 m^-2; phi = {phi}")
for (L0, DL, R, t) in source_sample:
    sigma = DL/FWHM_factor
    # print(thickness_map[(L0, R)])
    print(F"R = {R * 1e9}nm, t = {round(t * 1e3,2)}mm, lambda = {L0} Å (sigma = {sigma} Å):")
    st_0 = s_t(R,t, L0 * 1e-10, phi, delta_rho)
    T_0 = np.exp(-st_0)
    
    lambdas = np.linspace(L0 - 5 * sigma, L0 + 5 * sigma, 100000)
    dlambda = lambdas[1] - lambdas[0]
    p = norm.pdf(lambdas, L0, sigma)
    sts = s_t(R,t, lambdas * 1e-10, phi, delta_rho)
    st_integrand = sts * p 
    st_mean = integrate.trapezoid(st_integrand, x = lambdas)
    T_integrand = np.exp(-sts) * p 
    T_mean = integrate.trapezoid(T_integrand, x = lambdas)
    # plt.plot(x, p, linewidth=2)
    # plt.plot(x, integrand)
    print(f"\ts*t: {round(st_0, 4)}\n\t T: {round(T_0, 4)}\ts*t mean: {round(st_mean, 4)}\n\t T mean: {round(T_mean, 4)}")
    # sigma = s_t(R,t, L0 * 1e-10) / t
    # t_optimum = 0.38 / sigma

# $Q_{max}$ and $\lambda$ dependence? This could play a role but probably does not explain everything
Using the wavelengths $\lambda_0 - \Delta\lambda$ and $\lambda_0 + \Delta\lambda$ indicates that the $Q_{max}$ values for these differs by about $22$\% for the velocity selector, compared to only $2.02$\% for the PG monochromator.  

In [ ]:
for ratio in [0.01, 0.1]:
    max_factor = 1 / (1 - ratio)
    min_factor = 1 / (1 + ratio)
    print(min_factor, max_factor, max_factor / min_factor)

# Convergence of random search optimization

In [ ]:
import pickle 
import matplotlib.pyplot as plt
import numpy as np
# Open the file in binary mode 
with open('fit_stat_map.pkl', 'rb') as file: 
    # Call load method to deserialze 
    fit_stat_map = pickle.load(file) 


def plot_with_error_shaded(x,y,y_std,label):
    plt.plot(x, y, '.', label=label)
    plt.fill_between(x, y-y_std, y+y_std, alpha = 0.5)

for mc_type in ['PG', 'VS']:
    for prec_type in ['iso','foil', 'wsp','foil2',]:
        N_range, fitness_mean, fitness_std = fit_stat_map[(prec_type,mc_type)]
        N_start_ix = 25
        plot_with_error_shaded(N_range[N_start_ix:], fitness_mean[N_start_ix:], fitness_std[N_start_ix:], prec_type)
    plt.legend()
    plt.xlabel('N')
    plt.ylabel(r'$\bar{g}_3$')
    plt.title(r'$\bar{g}_3$ for' + f' {mc_type} instruments, average over 16 runs')
    plt.grid()
    plt.show()

N_range, fitness_mean, fitness_std = fit_stat_map[('foil2','PG')]
plot_with_error_shaded(N_range[N_start_ix:], fitness_mean[N_start_ix:], fitness_std[N_start_ix:], prec_type)
plt.legend()
plt.xlabel('N')
plt.ylabel(r'$\bar{g}_3$')
plt.title(r'$\bar{g}_3$ for FOIL2 PG, average over 16 runs')
plt.grid()
plt.show()